In [4]:
%matplotlib inline

import nltk.data

from gensim.models import word2vec
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree

import pandas as pd
import numpy as np

import os
import re
import logging;
import time
import sys
import multiprocessing
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
from itertools import cycle

In [5]:
import codecs

In [6]:
corpus = u""
book1 = codecs.open('got1.txt', 'r', "utf-8")
book2 = codecs.open('got2.txt', 'r', "utf-8")
book3 = codecs.open('got3.txt', 'r', "utf-8")
book4 = codecs.open('got4.txt', 'r', "utf-8")
book5 = codecs.open('got5.txt', 'r', "utf-8")

In [7]:
corpus += book1.read()
print(len(corpus), " after first book")
corpus += book2.read()
print(len(corpus), " after second book")
corpus += book3.read()
print(len(corpus), " after thirst book")
corpus += book4.read()
print(len(corpus), " after forth book")
corpus += book5.read()
print(len(corpus), " after fifth book")

1768659  after first book
4065904  after second book
5922012  after thirst book
7634356  after forth book
9243933  after fifth book


In [8]:
len(corpus)

9243933

In [9]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle');

In [10]:
raw_sentences = tokenizer.tokenize(corpus)

In [11]:
raw_sentences[6780]

'I’ve won no tourneys, no renown in war .'

In [12]:
#cleaning the corpus a bit
def sentence_to_wordlist(sentence):
    no_tabs = str(sentence).replace('\t', ' ').replace('\n', ' ');
    
        #Remove all characters except A-Z and a dot.
    alphas_only = re.sub("[^a-zA-Z\.]", " ", no_tabs);
        
        #Normalize spaces to 1
    multi_spaces = re.sub(" +", " ", alphas_only);
        
        #Strip trailing and leading spaces
    no_spaces = multi_spaces.strip();
        
        #Normalize all charachters to lowercase
    clean_text = no_spaces.lower();
    words = clean_text.split()
    return words

In [13]:
sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

In [14]:
print(sentences[3])
print(raw_sentences[3])

['their', 'droppings', 'speckled', 'the', 'gargoyles', 'that', 'rose', 'twelve', 'feet', 'tall', 'on', 'either', 'side', 'of', 'him', 'a', 'hellhound', 'and', 'a', 'wyvern', 'two', 'of', 'the', 'thousand', 'that', 'brooded', 'over', 'the', 'walls', 'of', 'the', 'ancient', 'fortress.']
Their droppings speckled the gargoyles that rose twelve feet tall on either side of him, a hellhound and a wyvern, two of the thousand that brooded over the walls of the ancient fortress.


In [15]:
start = time.time();

#Set the logging format to get some basic updates.
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Dimensionality of the hidden layer representation
min_word_count = 3   # Minimum word count to keep a word in the vocabulary
num_workers = multiprocessing.cpu_count();       # Number of threads to run in parallel set to total number of cpus.
context = 7          # Context window size (on each side)                                                       
downsampling = 1e-3   # Downsample setting for frequent words
seed=1
# Initialize and train the model. 
#The LineSentence object allows us to pass in a file name directly as input to Word2Vec,
#instead of having to read it into memory first.

print("Training model...");
#model = word2vec.Word2Vec(sentences, workers=num_workers, \
#            size=num_features, min_count = min_word_count, \
#            window = context, sample = downsampling);

model = word2vec.Word2Vec(sentences=sentences,
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context,
    sample=downsampling
)

# We don't plan on training the model any further, so calling 
# init_sims will make the model more memory efficient by normalizing the vectors in-place.
model.init_sims(replace=True);

# Save the model
model_name = "model_full_reddit";
model.save(model_name);

print('Total time: ' + str((time.time() - start)) + ' secs')


2018-06-24 23:27:59,594 : INFO : collecting all words and their counts
2018-06-24 23:27:59,596 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-24 23:27:59,651 : INFO : PROGRESS: at sentence #10000, processed 142074 words, keeping 11760 word types
2018-06-24 23:27:59,685 : INFO : PROGRESS: at sentence #20000, processed 282065 words, keeping 15922 word types
2018-06-24 23:27:59,714 : INFO : PROGRESS: at sentence #30000, processed 423382 words, keeping 18895 word types


Training model...


2018-06-24 23:27:59,743 : INFO : PROGRESS: at sentence #40000, processed 560641 words, keeping 21076 word types
2018-06-24 23:27:59,773 : INFO : PROGRESS: at sentence #50000, processed 691679 words, keeping 22830 word types
2018-06-24 23:27:59,812 : INFO : PROGRESS: at sentence #60000, processed 834169 words, keeping 24573 word types
2018-06-24 23:27:59,848 : INFO : PROGRESS: at sentence #70000, processed 979648 words, keeping 26358 word types
2018-06-24 23:27:59,881 : INFO : PROGRESS: at sentence #80000, processed 1120745 words, keeping 27782 word types
2018-06-24 23:27:59,910 : INFO : PROGRESS: at sentence #90000, processed 1251330 words, keeping 28969 word types
2018-06-24 23:27:59,940 : INFO : PROGRESS: at sentence #100000, processed 1385500 words, keeping 30077 word types
2018-06-24 23:27:59,973 : INFO : PROGRESS: at sentence #110000, processed 1533055 words, keeping 31108 word types
2018-06-24 23:28:00,006 : INFO : PROGRESS: at sentence #120000, processed 1688392 words, keeping 3

2018-06-24 23:28:27,677 : INFO : EPOCH 5 - PROGRESS: at 11.96% examples, 155545 words/s, in_qsize 15, out_qsize 0
2018-06-24 23:28:28,706 : INFO : EPOCH 5 - PROGRESS: at 25.84% examples, 165062 words/s, in_qsize 15, out_qsize 0
2018-06-24 23:28:29,724 : INFO : EPOCH 5 - PROGRESS: at 39.15% examples, 163957 words/s, in_qsize 14, out_qsize 1
2018-06-24 23:28:30,732 : INFO : EPOCH 5 - PROGRESS: at 52.30% examples, 165510 words/s, in_qsize 15, out_qsize 0
2018-06-24 23:28:31,735 : INFO : EPOCH 5 - PROGRESS: at 65.24% examples, 166779 words/s, in_qsize 15, out_qsize 0
2018-06-24 23:28:32,737 : INFO : EPOCH 5 - PROGRESS: at 81.89% examples, 172380 words/s, in_qsize 15, out_qsize 0
2018-06-24 23:28:33,779 : INFO : EPOCH 5 - PROGRESS: at 94.62% examples, 172318 words/s, in_qsize 11, out_qsize 0
2018-06-24 23:28:33,943 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-06-24 23:28:33,983 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-06-24 23:28:33,

Total time: 35.361692905426025 secs


In [16]:
Z = model.wv.syn0;

print(Z[0].shape)
Z[0]

(300,)


/home/therochvoices/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


array([ 6.43435195e-02, -4.41420497e-03,  1.09267542e-02, -5.28681502e-02,
        5.40726371e-02, -3.83774028e-03, -6.53521046e-02,  4.14840057e-02,
       -4.88582905e-03, -3.08059319e-03, -2.08482463e-02, -1.18899710e-01,
       -3.91015224e-02,  7.78779946e-03,  1.55423591e-02, -3.79761383e-02,
       -3.54082920e-02, -2.57708672e-02, -3.19017209e-02,  9.54599753e-02,
       -3.73223145e-03, -6.49367459e-03, -1.32358775e-01,  8.68466198e-02,
       -4.16426221e-03,  4.59575728e-02,  4.02442589e-02, -3.18773091e-02,
       -9.17632040e-03,  3.51376012e-02,  1.07867531e-02, -1.72238816e-02,
       -4.81199240e-03, -2.86696106e-02, -2.49801055e-02,  5.75697189e-03,
        5.04491031e-02,  3.40129286e-02, -5.90407513e-02, -7.18327053e-03,
        2.25798376e-02, -7.79880285e-02,  5.70933968e-02, -8.94143060e-02,
        4.15351614e-03,  7.93885887e-02, -5.37117347e-02,  2.51808297e-02,
       -1.63294412e-02,  6.34267600e-03, -4.33077477e-02, -5.68772890e-02,
        1.74077265e-02,  

In [17]:
print(Z.shape)

(18676, 300)


In [18]:
def clustering_on_wordvecs(word_vectors, num_clusters):
    # Initalize a k-means object and use it to extract centroids
    kmeans_clustering = KMeans(n_clusters = num_clusters, init='k-means++')
    idx = kmeans_clustering.fit_predict(word_vectors)
    
    return kmeans_clustering.cluster_centers_, idx

In [19]:
centers, clusters = clustering_on_wordvecs(Z, 50);
centroid_map = dict(zip(model.wv.index2word, clusters));

In [20]:
type(centroid_map)

dict

In [21]:
def get_top_words(index2word, k, centers, wordvecs):
    tree = KDTree(wordvecs);

#Closest points for each Cluster center is used to query the closest 20 points to it.
    closest_points = [tree.query(np.reshape(x, (1, -1)), k=k) for x in centers];
    closest_words_idxs = [x[1] for x in closest_points];

#Word Index is queried for each position in the above array, and added to a Dictionary.
    closest_words = {};
    for i in range(0, len(closest_words_idxs)):
        closest_words['Cluster #' + str(i)] = [index2word[j] for j in closest_words_idxs[i][0]]

#A DataFrame is generated from the dictionary.
    df = pd.DataFrame(closest_words);
    df.index = df.index+1

    return df;

In [22]:
top_words = get_top_words(model.wv.index2word, 2000, centers, Z);

In [23]:
top_words

,Cluster #0,Cluster #1,Cluster #10,Cluster #11,Cluster #12,Cluster #13,Cluster #14,Cluster #15,Cluster #16,Cluster #17,...,Cluster #45,Cluster #46,Cluster #47,Cluster #48,Cluster #49,Cluster #5,Cluster #6,Cluster #7,Cluster #8,Cluster #9
1,wobbly,shoot,burning.,fastenings.,ruling,grubbs,frankly.,unadorned.,godswife,kite,...,substitute,joke.,stairwell,gawen,blount.,northeast,punish,screeching.,feelings,horizon.
2,nipple.,across.,memory.,garnet,faith.,eustace,calls.,garnets.,daario.,fired.,...,mudmen,hopeful.,chorus.,tion,meryn.,northwest,aid,trophies.,wiser.,immensity
3,coiling,hang.,unseen,overlapping,subjects,respectively.,allowing,lavender,cedra,lumber,...,squabbling.,eyeing,alleys.,leobald,aron,bank.,shall.,shop.,proving,leafless
4,rubbing,sneak,shadows.,buttons,righteous,brax.,milady.,torn.,blushing.,mermen,...,salting,managed.,hut,berena,swann.,heights,test,stockade.,otherwise.,snowcapped
5,oozed,weese.,ashes.,medallions,friendship,mallery,concede,wire.,scolera,stonetrees,...,owned.,healer.,collapse.,beren,kevan.,shipbreaker,persuade,surrender.,chosen.,soared
6,wetness,stumble,burned.,vair.,reconcile,menagerie.,prove.,supple.,irri.,inferno,...,impossible.,lust.,sleeper,hosman,andrew,foothills,willingly,clamor.,doubted,between.
7,flailed,trail.,confusion,miniature,woes,emrick.,beggar.,script,jhiqui.,structures,...,belis,grandchildren.,refuge.,minisa,lyle,watchtower,shames,reported.,bothered,festooned
8,thigh.,heal.,depths,calfskin,forsaken,gorys,provocation.,arrowheads.,qezza,skane,...,baser,awaken.,sleet,westerling,brus,paths,condemn,eats.,amusing.,pocked
9,wincing,hell.,bright.,artfully,cause.,celladar,randa.,cap.,missandei.,lengths,...,ramming,minded.,dim.,evenstar,horpe,rooftops,prevail,rounsey.,secret.,windswept
10,baring,stragglers,dusk.,feathers.,trade.,cowardice.,disarm,moons.,maegi,hulls.,...,horsemeat.,mathos,seats.,cerenna,swyft.,summit,fix,pushing.,asking.,sod


In [24]:
def print_word_table(table, key):
    return pd.DataFrame(table, columns=[key, 'similarity'])

print_word_table(model.wv.most_similar_cosmul(positive=['king', 'woman'], negative=['queen']), 'Analogy')

,Analogy,similarity
0,man,0.849912
1,lordling,0.844931
2,craven,0.837408
3,thief,0.828095
4,northman,0.822078
5,presumed,0.820934
6,woman.,0.818246
7,dalla,0.817982
8,outlaw,0.817880
9,wolf.,0.817286


In [25]:
model.wv.doesnt_match("arya sansa jon rickon bran robb".split())

'jon'

In [26]:
model.most_similar("stark")

/home/therochvoices/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('stark.', 0.8070828318595886),
 ('eddard', 0.7496935725212097),
 ('robb', 0.6769108772277832),
 ('winterfell', 0.6614609360694885),
 ('beheaded', 0.6558789014816284),
 ('direwolf', 0.6514153480529785),
 ('benjen', 0.644086480140686),
 ('boltons', 0.6330555081367493),
 ('winterfell.', 0.6272383332252502),
 ('roslin', 0.6243686676025391)]

In [27]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = model.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{start1} is related to {end1}, as {start2} is related to {end2}".format(**locals()))
    return start2

In [28]:
nearest_similarity_cosmul("stark", "winterfell", "riverrun")
nearest_similarity_cosmul("jaime", "sword", "wine")
nearest_similarity_cosmul("arya", "nymeria", "dragons")

stark is related to winterfell, as tully is related to riverrun
jaime is related to sword, as tyrion is related to wine
arya is related to nymeria, as dany is related to dragons


/home/therochvoices/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
  after removing the cwd from sys.path.


'dany'

In [29]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [30]:
top_words = get_top_words(model.wv.index2word, 20, centers, Z);

In [31]:
top_words

,Cluster #0,Cluster #1,Cluster #10,Cluster #11,Cluster #12,Cluster #13,Cluster #14,Cluster #15,Cluster #16,Cluster #17,Cluster #18,Cluster #19,Cluster #2,Cluster #20,Cluster #21,Cluster #22,Cluster #23,Cluster #24,Cluster #25,Cluster #26,Cluster #27,Cluster #28,Cluster #29,Cluster #3,Cluster #30,Cluster #31,Cluster #32,Cluster #33,Cluster #34,Cluster #35,Cluster #36,Cluster #37,Cluster #38,Cluster #39,Cluster #4,Cluster #40,Cluster #41,Cluster #42,Cluster #43,Cluster #44,Cluster #45,Cluster #46,Cluster #47,Cluster #48,Cluster #49,Cluster #5,Cluster #6,Cluster #7,Cluster #8,Cluster #9
1,wobbly,shoot,burning.,fastenings.,ruling,grubbs,frankly.,unadorned.,godswife,kite,unworthy.,asking.,kinslaying.,tension.,flying.,halted,ages,thread,clumsily,qarth.,apples.,melesa,widening,hiss,sloppy,appointments,teams,wiser.,foolish.,deadfalls,carefully.,disciplined,grip.,nephew.,ebben,horrible,butt.,owns,lank,greyjoys,substitute,joke.,stairwell,gawen,blount.,northeast,punish,screeching.,feelings,horizon.
2,nipple.,across.,memory.,garnet,faith.,eustace,calls.,garnets.,daario.,fired.,edrick,lose.,ridding,panic.,across.,retreated,veteran,mockingbird,ducked,tolos,oaten,sallei,ducking,whistling,gagged,henceforth.,jostling,maiming,squalling,bluff.,sweetly.,soldiers.,blade.,consent.,hake,ashamed.,armpits.,outlanders,eyebrows,unleashed,mudmen,hopeful.,chorus.,tion,meryn.,northwest,aid,trophies.,wiser.,immensity
3,coiling,hang.,unseen,overlapping,subjects,respectively.,allowing,lavender,cedra,lumber,domeric,tried.,stock.,carries,manacled,pacing,votes,thread.,broomstick,slavery,biscuits,sylwa,bodily,footfalls,bladder,occupation,dozens,temptation,solace.,courtyards.,acting,crews,hilt.,acknowledged,rattleshirt.,sobbing.,rattling.,peoples.,splotchy,stormed,squabbling.,eyeing,alleys.,leobald,aron,bank.,shall.,shop.,proving,leafless
4,rubbing,sneak,shadows.,buttons,righteous,brax.,milady.,torn.,blushing.,mermen,strangled,lied.,wylla.,bristle.,nymeria.,slept.,five.,embroidered,backwards,magisters,rasher,william,surf,crackle,sobbing.,rightly,firing,discourteous,clever.,shallows.,astonished.,weakest,dirk.,conspired,reysen,despair.,effortlessly,helping.,leathery,seagard.,salting,managed.,hut,berena,swann.,heights,test,stockade.,otherwise.,snowcapped
5,oozed,weese.,ashes.,medallions,friendship,mallery,concede,wire.,scolera,stonetrees,childless,mance.,irritably.,shove.,garron.,forcefully,weddings,device,tripped,revolt,fowl,hornvale,punching,echoes,snowball,betrays,tents.,act.,handsome.,pond.,shyly.,skinchangers,morningstar,marrying,halfhand.,dreaming.,canter.,equipped,windburnt,forces,owned.,healer.,collapse.,beren,kevan.,shipbreaker,persuade,surrender.,chosen.,soared
6,wetness,stumble,burned.,vair.,reconcile,menagerie.,prove.,supple.,irri.,inferno,governance,freely.,especial,amabel.,wallwalk,explored,ten.,burnished,thumped,goddess,bread.,hawick,gunwale,sighing,tummy,forgiving,tethered,precious.,maiden.,aisle.,unhappily.,raping,greatsword.,betrothal,scowling.,weeping.,flank.,bargain.,knobby,summerhall,impossible.,lust.,sleeper,hosman,andrew,foothills,willingly,clamor.,doubted,between.
7,flailed,trail.,confusion,miniature,woes,emrick.,beggar.,script,jhiqui.,structures,deceased,banish,challenger,critically.,koss,conducted,eighty,bat,sideways.,pureborn,tarts,jyanna,trunk.,groaning,ache.,intentions,warhorses,elsewise.,suspicious.,sheets.,tensed.,thousands.,lodged,wedded,bannen,listened.,released.,loving.,pug,stepstones.,belis,grandchildren.,refuge.,minisa,lyle,watchtower,shames,reported.,bothered,festooned
8,thigh.,heal.,depths,calfskin,forsaken,gorys,provocation.,arrowheads.,qezza,skane,sunderland,should.,toast.,rebuke.,angrily,breathed,elder.,blazoned,undid,nobles,butter.,blanetree,sill,shrieks,sob.,consort.,crowding,proof.,freakish,thud,pointedly.,fighters,groin,lyanna,farlen,suspicion,unceremoniously,hazzat,squashed,governance,baser,awaken.,sleet,westerling,brus,paths,condemn,eats.,amusing.,pocked
9,wincing,hell.,bright.,artfully,cause.,celladar,randa.,cap.,missan

In [32]:
def display_cloud(cluster_num, cmap):
    wc = WordCloud(background_color="black", max_words=2000, max_font_size=80, colormap=cmap);
    wordcloud = wc.generate(' '.join([word for word in top_words['Cluster #' + str(cluster_num).zfill(2)]]))

    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.savefig('cluster_' + str(cluster_num), bbox_inches='tight')

In [34]:
'''cmaps = cycle([
            'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
            'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg', 'hsv',
            'gist_rainbow', 'rainbow', 'jet', 'nipy_spectral', 'gist_ncar'])

for i in range(50):
    col = next(cmaps);
    display_cloud(i+1, col)'''

"cmaps = cycle([\n            'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',\n            'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg', 'hsv',\n            'gist_rainbow', 'rainbow', 'jet', 'nipy_spectral', 'gist_ncar'])\n\nfor i in range(50):\n    col = next(cmaps);\n    display_cloud(i+1, col)"

In [35]:
#visualizing the vectors as taught by Siraj Rawal
import sklearn.manifold
tsne = sklearn.manifold.TSNE(n_components=2, random_state=0)
all_word_vectors_matrix_2d = tsne.fit_transform(Z)

In [36]:
points = pd.DataFrame(
    [
        (word, coords[0], coords[1])
        for word, coords in [
            (word, all_word_vectors_matrix_2d[model.wv.vocab[word].index])
            for word in model.wv.vocab
        ]
    ],
    columns=["word", "x", "y"]
)

In [37]:
points.head(10)

,word,x,y
0,the,13.501125,22.189024
1,comet,37.751148,-34.086235
2,s,-34.989349,34.409000
3,tail,26.407074,-50.823784
4,spread,29.423889,-35.453869
5,across,26.624086,-32.476799
6,dawn,-23.405998,-27.023907
7,a,-15.723437,-24.206289
8,red,52.686161,-24.737574
9,slash,16.953718,-50.210819


In [38]:
import seaborn as sns
sns.set_context("poster")

In [47]:
%matplotlib
plt.ioff()
points.plot.scatter("x", "y", s=10, figsize=(20, 12))

fig, ax = plt.subplots()
for i, txt in enumerate(points['word']):
    ax.annotate(txt, (points['x'][i],points['y'][i]))
    
plt.show()

Using matplotlib backend: Qt5Agg


In [40]:

#for i, txt in enumerate(points['word']):
#    print(txt, i)
#    ax.annotate(txt, (x[i],y[i]))
print(points['x'][0])

13.50112533569336
